#Hashing with Linear Probing

In [ ]:
class Dictionary:
  def __init__(self, size):
    self.size = size
    self.slots = [None] * self.size
    self.data = [None] * self.size

  def hash_function(self, key):
    return abs(hash(key)) % self.size

  def rehash(self, old_hash):
    return (old_hash + 1) % self.size

  def put(self, key, val):
    hash_val = self.hash_function(key)

    if self.slots[hash_val] == None:
      self.slots[hash_val] = key
      self.data[hash_val] = val
    else:
      if self.slots[hash_val] == key:
        #key already exists
        self.dat[hash_val] == val

      else:
        #collision occured
        new_hash_val = self.rehash(hash_val)

        #keep shifting using hash func until,
          # - an empty slot is found
          # - key is found
        while self.slots[new_hash_val] != None and self.slots[new_hash_val] != key:
          new_hash_val = self.rehash(new_hash_val)

        if self.slots[new_hash_val] == None:
          self.slots[new_hash_val] = key
          self.data[new_hash_val] = val
        else:
          self.data[new_hash_val] = val


  def get(self, key):
    start = self.hash_function(key)
    curr = start

    while self.slots[curr] != None:
      if self.slots[curr] == key:
        return self.data[curr]

      curr = self.rehash(curr)

      if curr == start:
        #1 cycle completed
        return 'Not Found'

    return 'Not Found'

  def __setitem__(self, key, val):
    self.put(key, val)

  def __getitem__(self, key):
    return self.get(key)

  def __str__(self):
    for i in range(self.size):
      if self.slots[i] != None:
        print(self.slots[i], ':', self.data[i], end=', ')

    return ""

In [ ]:
d1 = Dictionary(5)

In [ ]:
print(d1.slots)
print(d1.data)

[None, None, None, None, None]
[None, None, None, None, None]


In [ ]:
d1.put('python', 100)
d1.put('java', 150)
d1.put('c', 300)

print(d1.slots)
print(d1.data)

['c', 'java', 'python', None, None]
[300, 150, 100, None, None]


In [ ]:
d1['php'] = 50
d1['javascript'] = 80

print(d1.slots)
print(d1.data)

['c', 'java', 'python', 'php', 'javascript']
[300, 150, 100, 50, 80]


In [ ]:
d1.get('python')

100

In [ ]:
d1.get('dummy')

'Not Found'

In [ ]:
d1['javascript']

80

In [ ]:
d1['dummy']

'Not Found'

In [ ]:
print(d1)

c : 300, java : 150, python : 100, php : 50, javascript : 80, 


#Hashing using Chaining

In [ ]:
class Node:
  def __init__(self, key, val):
    self.key = key
    self.val = val
    self.next = None

In [ ]:
class LL:
  #Array of LinkedLists
  def __init__(self):
    #empty LL
    self.head = None
    self.n = 0


  ### INSERTION METHODS

  def add(self, key, val):
    new_node = Node(key, val)
    if self.head == None:
      #empty list
      self.head = new_node
      self.n += 1
    else:
      #init pointer
      curr = self.head
      #shift pointer until last node
      while curr.next != None:
        curr = curr.next
      #assign last pointer to new node
      curr.next = new_node
      self.n += 1


  ### DELETION METHODS

  def delete_head(self):
    if self.head == None:
      return 'Empty List'
    self.head = self.head.next
    self.n = self.n - 1


  def remove(self, key):
    if self.head.key == key:
      self.delete_head()
      return

    if self.head == None:
      #empty list
      return 'Empty LL'
    else:
      curr = self.head

      while curr.next != None:
        if curr.next.key == key:
          curr.next = curr.next.next
          self.n -= 1
          return
        curr = curr.next

    return 'Key not found'


  ### SEARCH METHODS
  def search(self, key):
    curr = self.head
    pos = 0

    while curr != None:
      if curr.key == key:
        return pos
      curr = curr.next
      pos += 1

    return -1

  def get_node_at_idx(self, idx):
    curr = self.head
    counter = 0

    while curr != None:
      if counter == idx:
        return curr
      curr = curr.next
      counter += 1


  ### MAGIC METHODS
  def __len__(self):
    return self.n


  def __str__(self):
    #init pointer
    curr = self.head
    result = ''

    #shift pointer until last node
    while curr != None:
      print(curr.key, '-->', curr.val, ' ', end=' ')
      #move pointer
      curr = curr.next
    return ""


  def __getitem__(self, idx):
    curr = self.head
    pos = 0

    while curr != None:
      if pos == idx:
        return curr.data
      curr = curr.next
      pos += 1

    return 'IndexError'


In [ ]:
class DictionaryChaining:
  def __init__(self, capacity):
    self.capacity = capacity
    self.size = 0
    #create buckets -> Array of LLs
    # - each bucket -> LL
    self.buckets = self.make_array(self.capacity)

  def make_array(self, capacity):
    L = []
    for i in range(capacity):
      L.append(LL())
    return L

  def put(self, key, val):
    bucket_idx = self.hash_function(key)

    node_idx = self.get_node_idx(bucket_idx, key)

    if node_idx == -1:
      #insert
      bucket = self.buckets[bucket_idx]
      bucket.add(key, val)
      self.size += 1

      load_factor = self.size / self.capacity

      if load_factor >= 2:
        self.rehash()

    else:
      #update
      bucket = self.buckets[bucket_idx]
      node = bucket.get_node_at_idx(node_idx)
      node.val = val


  def delete(self, key):
    bucket_idx = self.hash_function(key)

    self.buckets[bucket_idx].remove(key)
    self.size -= 1

    if self.size < self.capacity:
      self.rehash_downsize()


  def get(self, key):
    bucket_idx = self.hash_function(key)

    res = self.buckets[bucket_idx].search(key)

    if res == -1:
      return -1
    else:
      node = self.buckets[bucket_idx].get_node_at_idx(res)
      return node.val


  def get_node_idx(self, bucket_idx, key):
    node_idx = self.buckets[bucket_idx].search(key)
    return node_idx


  def hash_function(self, key):
    return abs(hash(key)) % self.capacity


  def rehash(self):
    self.capacity = self.capacity*2
    old_buckets = self.buckets
    self.size = 0
    self.buckets = self.make_array(self.capacity)

    for bucket in old_buckets:
      for j in range(len(bucket)):
        node = bucket.get_node_at_idx(j)
        key_item = node.key
        val_item = node.val
        self.put(key_item, val_item)


  def rehash_downsize(self):
    if self.size == 0:
      #zero nodes
      return

    self.capacity = self.capacity//2

    old_buckets = self.buckets
    self.size = 0
    self.buckets = self.make_array(self.capacity)

    for bucket in old_buckets:
      for j in range(len(bucket)):
        node = bucket.get_node_at_idx(j)
        key_item = node.key
        val_item = node.val
        self.put(key_item, val_item)



  def __setitem__(self, key, val):
    return self.put(key, val)


  def __getitem__(self, key):
    return self.get(key)


  def __delitem__(self, key):
    return self.delete(key)


  def __len__(self):
    return self.size


  def __str__(self):
    for i in range(len(d2.buckets)):
      print(f"Bucket {i}:")
      print(self.buckets[i], '\n')
    return ""

In [ ]:
d2 = DictionaryChaining(2)

In [ ]:
d2.put('python', 32)
d2.put('java', 20)
d2.put('c', 100)

In [ ]:
d2.buckets

[<__main__.LL at 0x7a1e15e92c80>, <__main__.LL at 0x7a1e15e93220>]

In [ ]:
print(d2)

Bucket 0:
python --> 32   java --> 20   c --> 100    

Bucket 1:
 




In [ ]:
len(d2)

3

In [ ]:
d2.put('ruby', 15)

In [ ]:
d2.buckets

In [ ]:
print(d2)

Bucket 0:
python --> 32   java --> 20    

Bucket 1:
ruby --> 15    

Bucket 2:
c --> 100    

Bucket 3:
 




In [ ]:
len(d2)

4

In [ ]:
d2['java']

20

In [ ]:
d2['abcd']

-1

In [ ]:
del d2['java']
print(d2)

Bucket 0:
python --> 32   c --> 100    

Bucket 1:
ruby --> 15    




In [ ]:
d2['java']

-1

In [ ]:
len(d2)

3

In [ ]:
del d2['ruby']
print(d2)

Bucket 0:
python --> 32   c --> 100    

Bucket 1:
 




In [ ]:
len(d2)

2

In [ ]:
del d2['c']
print(d2)

Bucket 0:
python --> 32    




In [ ]:
del d2['python']
print(d2)

Bucket 0:
 




In [ ]:
d2.put('c++', 300)
print(d2)

Bucket 0:
c++ --> 300    




In [ ]:
d2.put('excel', 10)
d2.put('sql', 30)
d2.put('pytorch', 500)
print(d2)

Bucket 0:
c++ --> 300   sql --> 30    

Bucket 1:
excel --> 10   pytorch --> 500    

Bucket 2:
 

Bucket 3:
 


